# Automated Model Testing 

This notebook contains a set of automated tests for the CCU model.  These tests are either pass or fail and no interpretation is needed. A summary of test results is provided at the end of the notebook.

## Imports

In [1]:
import numpy as np
import statistics
from sim_tools.distributions import Lognormal
import pytest
import ipytest
ipytest.autoconfig()

## Model Code Imports

In [2]:
from ccu_formatted_code import *

## Tests

### Constants and utility functions

In [3]:
# The extreme value
M = 10_000_000

In [4]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    try:
        results = ccu_model.run()
    except ZeroDivisionError:
        results = None
    
    return ccu_model, results

### Extreme value test: no elective arrivals

In [5]:
def test_extreme_value_no_electives(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time
    set to $M$ a very large number.
    
    Expected result: 
    ----------------
    Quantitative: There are no elective cancellations (as no electives arrive)

    See also manual version:
    The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    model, results =  run_test(experiment)

    # test elective count should = 0
    assert results['Cancelled Elective Operations'].iloc[0] == 0

### Extreme value test: 1 critical care bed available

In [6]:
@pytest.mark.parametrize('random_number_set', [
                          (42),(101),(1),(2),(1234)
])
def test_extreme_value_1_bed(random_number_set):
    '''
    Extreme value test: Critical care beds set to 1 
    
    Expected result: 
    ---------------
    bed occupancy to be equal to bed utilization as only 1 bed is available
    
    Cancellations also begin after 1st arrival.
 
    Note: 
    ----
    [1] When critical_care_beds=1 queues form after first arrival.  This is
    assessed manually by viewing the trace in the manual trace notebook.

    [2] Simpy constraints force us to use 1 bed instead of 0.
    
    '''
    experiment = Experiment(num_critical_care_beds=1, 
                            random_number_set=random_number_set)
    model, results = run_test(experiment)
    assert results['Bed Utilization'].iloc[0] == results['Bed Occupancy'].iloc[0]

### Extreme value test: infinite critical care capacity.

In [7]:
@pytest.mark.parametrize('random_number_set', [
                          (42),(101),(1),(2),(1234)
])
def test_extreme_value_infinite_capacity(random_number_set):
    '''
    Extreme value test 3: 
    
    Critical care beds set to M a very large number.
    
    Expected result: 
    ---------------
    No cancelled electives and no unplanned patients have to wait.    
    '''
    experiment = Experiment(num_critical_care_beds=M, 
                            random_number_set=random_number_set)
    model, results = run_test(experiment)
    assert results['Cancelled Elective Operations'].iloc[0] == 0 and \
           results['Mean Waiting Time Unplanned'].iloc[0] == 0

### Test warm-up period reset

In [8]:
@pytest.mark.parametrize('warm_up', [
                          (0),(1),(100),(570),(1000)
])
def test_warmup_period(warm_up):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result
    ---------------
    Run length = warm_up + results collection period
    '''
    experiment = Experiment(warm_up_period=warm_up)
    model, results = run_test(experiment)
    assert model.env.now == (experiment.results_collection_period \
                             + experiment.warm_up_period)

In [9]:
@pytest.mark.parametrize('warm_up', [
                          (0),(1),(100),(570),(1000)
])
def test_warmup_reset(warm_up):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result
    ---------------
    The following `experiment` variables are set to 0:
    
    total_treatment_time
    cancelled_elective_count
    mean_waiting_time_unplanned
    total_unplanned_admissions

    The following model variables is reset to 0 during 
    warm_up_complete event:
    
    patient_count

    Notes:
    ------
    [1] We must set results collectiom period to a small
    number. This allows for the warm_up_complete event
    to take place before simpy terminates the run.
    
    '''
    # allow for very small results collection period so
    # that warmup reset event occurs.
    experiment = Experiment(warm_up_period=warm_up,
                            results_collection_period=0.005)
    
    model, results = run_test(experiment)

    assert experiment.total_treatment_time == 0 and \
           experiment.cancelled_elective_count == 0 and \
           experiment.mean_waiting_time_unplanned == 0 and \
           experiment.total_unplanned_admissions == 0 and \
           model.patient_count == 0

### Test repeatable runs

In [10]:
@pytest.mark.parametrize('n_reps', [
                          (5),(10),(27)
])
def test_repeatable_reps(n_reps):
    """
    Test that random number streams are controlled and 
    multiple replications produce the same results each
    time they are run.

    Params:
    -------
    n_reps: int
        The number of replications to run.

    Expected results:
    -----------------
    The difference of two repeated runs is 0.
    """
    experiment = Experiment()
    replications = multiple_replications(experiment, n_reps)
    rs1 =  results_summary(replications)

    experiment = Experiment()
    replications = multiple_replications(experiment, n_reps)
    rs2 =  results_summary(replications)

    # sum all performance measures -> if no diff then = 0
    diff = (rs1 - rs2).sum(axis=1).sum()

    assert diff == 0.0

## Run tests

In [11]:
ipytest.run("-vv", "--no-header", "--cov=ccu_formatted_code")

======================================= test session starts ========================================
collecting ... collected 24 items

t_460250ab3a9e445ea23c6442cf9e2228.py::test_extreme_value_no_electives PASSED                [  4%]
t_460250ab3a9e445ea23c6442cf9e2228.py::test_extreme_value_1_bed[42] PASSED                   [  8%]
t_460250ab3a9e445ea23c6442cf9e2228.py::test_extreme_value_1_bed[101] PASSED                  [ 12%]
t_460250ab3a9e445ea23c6442cf9e2228.py::test_extreme_value_1_bed[1] PASSED                    [ 16%]
t_460250ab3a9e445ea23c6442cf9e2228.py::test_extreme_value_1_bed[2] PASSED                    [ 20%]
t_460250ab3a9e445ea23c6442cf9e2228.py::test_extreme_value_1_bed[1234] PASSED                 [ 25%]
t_460250ab3a9e445ea23c6442cf9e2228.py::test_extreme_value_infinite_capacity[42] PASSED       [ 29%]
t_460250ab3a9e445ea23c6442cf9e2228.py::test_extreme_value_infinite_capacity[101] PASSED      [ 33%]
t_460250ab3a9e445ea23c6442cf9e2228.py::test_extreme_value_infini

<ExitCode.OK: 0>